In [1]:
import os
import numpy as np
import pandas as pd

In [8]:
DATA_DIR = os.getcwd()+'/Directivity/'
DIRECTIVITY_ANNOTATION = os.path.join(DATA_DIR, "MatrixLabelsDirectivity.csv")

df = pd.read_csv(DIRECTIVITY_ANNOTATION, delimiter=";")
df.head(5)

/home/lake/sguajardo/SoundOrientation/Directivity/


,audio_filename,class,event x (m),event y (m),event z (m),event orientation (x),event orientation (y),event orientation (z),listener (x),listener (y),listener (z),listener orientation (x),listener orientation (y),listener orientation (z),distance (m)
0,directivityEval00001.wav,Speech,7.8,1.8,-5.0,-1,0,0,5,1.8,-5,0,0,-1,2.8
1,directivityEval00002.wav,Speech,5.0,1.8,-7.7,0,0,-1,5,1.8,-5,0,0,-1,2.7
2,directivityEval00003.wav,Speech,7.3,1.8,-5.0,1,0,0,5,1.8,-5,0,0,-1,2.3
3,directivityEval00004.wav,Speech,8.1,1.8,-5.0,-1,0,0,5,1.8,-5,0,0,-1,3.1
4,directivityEval00005.wav,Speech,2.9,1.8,-5.0,1,0,0,5,1.8,-5,0,0,-1,2.1


In [3]:
df["distance (m)"].min()

1.5

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   audio_filename            8000 non-null   object 
 1   class                     8000 non-null   object 
 2   event x (m)               8000 non-null   float64
 3   event y (m)               8000 non-null   float64
 4   event z (m)               8000 non-null   float64
 5   event orientation (x)     8000 non-null   int64  
 6   event orientation (y)     8000 non-null   int64  
 7   event orientation (z)     8000 non-null   int64  
 8   listener (x)              8000 non-null   int64  
 9   listener (y)              8000 non-null   float64
 10  listener (z)              8000 non-null   int64  
 11  listener orientation (x)  8000 non-null   int64  
 12  listener orientation (y)  8000 non-null   int64  
 13  listener orientation (z)  8000 non-null   int64  
 14  distance

In [9]:
def cartesian2polar(event_x, event_z, listener_x0 = 0.0, listener_z0 = 0.0):
    x = event_x - listener_x0
    z = event_z - listener_z0
    degree = np.rad2deg(np.arctan2(z, x))
    r = np.sqrt(z**2 + x**2)
    return r, int(np.round(degree))

def get_categories(event_x, event_z, listener_x0, listener_z0, event_orientation_x, event_orientation_z):
    _, t = cartesian2polar(event_x, event_z, listener_x0, listener_z0)
    if t == 0:
        if event_orientation_x == -1:
            return {"l_0":0, "l_1":0, "l_2":0, "l_3":0, "l_4":0, "l_5":0, "l_6":0, "l_7":1}
        elif event_orientation_x == 1:
            return {"l_0":0, "l_1":0, "l_2":0, "l_3":0, "l_4":0, "l_5":0, "l_6":1, "l_7":0}
    elif t == 90:
        if event_orientation_z == -1:
            return {"l_0":0, "l_1":0, "l_2":0, "l_3":0, "l_4":0, "l_5":1, "l_6":0, "l_7":0}
        elif event_orientation_z == 1:
            return {"l_0":0, "l_1":0, "l_2":0, "l_3":0, "l_4":1, "l_5":0, "l_6":0, "l_7":0}
    elif t == 180:
        if event_orientation_x == -1:
            return {"l_0":0, "l_1":0, "l_2":0, "l_3":1, "l_4":0, "l_5":0, "l_6":0, "l_7":0}
        elif event_orientation_x == 1:
            return {"l_0":0, "l_1":0, "l_2":1, "l_3":0, "l_4":0, "l_5":0, "l_6":0, "l_7":0}
    elif t == -90:
        if event_orientation_z == -1:
            return {"l_0":0, "l_1":1, "l_2":0, "l_3":0, "l_4":0, "l_5":0, "l_6":0, "l_7":0}
        elif event_orientation_z == 1:
            return {"l_0":1, "l_1":0, "l_2":0, "l_3":0, "l_4":0, "l_5":0, "l_6":0, "l_7":0}
    else:
        print("Clase no identificada")
    

In [10]:
coordinates = []
for index,row in df.iterrows():
    r,t = cartesian2polar(row["event x (m)"], 
                          row["event z (m)"], 
                          row["listener (x)"], 
                          row["listener (z)"])
    coordinates.append({"distance":r, "theta":t})

df_coords = pd.DataFrame.from_dict(coordinates)
df_coords.head()

,distance,theta
0,2.8,0
1,2.7,-90
2,2.3,0
3,3.1,0
4,2.1,180


In [11]:
print(df_coords["theta"].value_counts())

theta
 0      3419
 90     3274
-90     1752
 180    1555
Name: count, dtype: int64


In [12]:
categories = []
for index,row in df.iterrows():
    
    categories.append(get_categories(
        row["event x (m)"], 
        row["event z (m)"], 
        row["listener (x)"], 
        row["listener (z)"], 
        row["event orientation (x)"],
        row["event orientation (z)"]
    ))

df_categories = pd.DataFrame.from_dict(categories)
df_categories.head()

,l_0,l_1,l_2,l_3,l_4,l_5,l_6,l_7
0,0,0,0,0,0,0,0,1
1,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,1
4,0,0,1,0,0,0,0,0


In [13]:
df_theta_categories = pd.concat([df_coords["theta"],df_categories], axis=1)
df_theta_categories.head()

,theta,l_0,l_1,l_2,l_3,l_4,l_5,l_6,l_7
0,0,0,0,0,0,0,0,0,1
1,-90,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,1
4,180,0,0,1,0,0,0,0,0


In [14]:
df_ = pd.concat([df,df_theta_categories], axis=1)
df_.head()

,audio_filename,class,event x (m),event y (m),event z (m),event orientation (x),event orientation (y),event orientation (z),listener (x),listener (y),...,distance (m),theta,l_0,l_1,l_2,l_3,l_4,l_5,l_6,l_7
0,directivityEval00001.wav,Speech,7.8,1.8,-5.0,-1,0,0,5,1.8,...,2.8,0,0,0,0,0,0,0,0,1
1,directivityEval00002.wav,Speech,5.0,1.8,-7.7,0,0,-1,5,1.8,...,2.7,-90,0,1,0,0,0,0,0,0
2,directivityEval00003.wav,Speech,7.3,1.8,-5.0,1,0,0,5,1.8,...,2.3,0,0,0,0,0,0,0,1,0
3,directivityEval00004.wav,Speech,8.1,1.8,-5.0,-1,0,0,5,1.8,...,3.1,0,0,0,0,0,0,0,0,1
4,directivityEval00005.wav,Speech,2.9,1.8,-5.0,1,0,0,5,1.8,...,2.1,180,0,0,1,0,0,0,0,0


In [24]:
print(df_["l_0"].value_counts())
print(df_["l_1"].value_counts())
print(df_["l_2"].value_counts())
print(df_["l_3"].value_counts())
print(df_["l_4"].value_counts())
print(df_["l_5"].value_counts())
print(df_["l_6"].value_counts())
print(df_["l_7"].value_counts())

l_0
0    5397
1     771
Name: count, dtype: int64
l_1
0    5397
1     771
Name: count, dtype: int64
l_2
0    5397
1     771
Name: count, dtype: int64
l_3
0    5397
1     771
Name: count, dtype: int64
l_4
0    5397
1     771
Name: count, dtype: int64
l_5
0    5397
1     771
Name: count, dtype: int64
l_6
0    5397
1     771
Name: count, dtype: int64
l_7
0    5397
1     771
Name: count, dtype: int64


In [23]:
count = 0
max_count = 771
for index, row in df_.iterrows():
    if row["l_7"] == 1:
        count+=1
        if count > max_count:
            df_ = df_.drop(index)

In [27]:
df_.to_csv(DATA_DIR + "MatrixLabelsDirectivityCategoriesBalanced.csv")

In [26]:
df_

,audio_filename,class,event x (m),event y (m),event z (m),event orientation (x),event orientation (y),event orientation (z),listener (x),listener (y),...,distance (m),theta,l_0,l_1,l_2,l_3,l_4,l_5,l_6,l_7
0,directivityEval00001.wav,Speech,7.8,1.8,-5.0,-1,0,0,5,1.8,...,2.8,0,0,0,0,0,0,0,0,1
1,directivityEval00002.wav,Speech,5.0,1.8,-7.7,0,0,-1,5,1.8,...,2.7,-90,0,1,0,0,0,0,0,0
2,directivityEval00003.wav,Speech,7.3,1.8,-5.0,1,0,0,5,1.8,...,2.3,0,0,0,0,0,0,0,1,0
3,directivityEval00004.wav,Speech,8.1,1.8,-5.0,-1,0,0,5,1.8,...,3.1,0,0,0,0,0,0,0,0,1
4,directivityEval00005.wav,Speech,2.9,1.8,-5.0,1,0,0,5,1.8,...,2.1,180,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9971,directivityEval09972.wav,Speech,2.9,1.8,-5.0,-1,0,0,5,1.8,...,2.1,180,0,0,0,1,0,0,0,0
9973,directivityEval09974.wav,Speech,1.9,1.8,-5.0,-1,0,0,5,1.8,...,3.1,180,0,0,0,1,0,0,0,0
9975,directivityEval09976.wav,Speech,2.5,1.8,-5.0,-1,0,0,5,1.8,...,2.5,180,0,0,0,1,0,0,0,0
9990,directivityEval09991.wav,Speech,1.8,1.8,-5.0,-1,0,0,5,1.8,...,3.2,180,0,0,0,1,0,0,0,0


In [23]:
df_.iloc[0, 0]

'directivityEval00001.wav'

In [24]:
os.path.join(DATA_DIR, df_.iloc[0, 0])

'/home/lake/sguajardo/SoundOrientation/Data_directivity/directivityEval00001.wav'